In [1]:
import tensorflow as tf
import numpy as np
from pathlib import Path
import gzip
import os

2023-03-10 14:20:37.973311: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 14:20:38.664157: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /data/dist/plumed2/src/lib/:/opt/gromacs/lib:/home/dima/miniconda3/envs/tf/lib/
2023-03-10 14:20:38.664227: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /data/dist/plumed2/src/lib/:/opt/gromacs/l

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
from run_opts import config_runtime
from src.logger import Logger
from dataset import read_list

# Read and prepare

In [29]:
n_cls = 3  # number of classes
# conf_dev = config_runtime['device']
train_f = config_runtime['train_frac']
seed = config_runtime['seed']
learning_rate = config_runtime['learning_rate']
batch_size = config_runtime['batch_size']
hid_size = config_runtime['hidden_size']
log_step = config_runtime['log_step']
epochs = config_runtime['num_epochs']
run_name = config_runtime['run_name']
hid_size = 32

In [30]:
logger = Logger("./", "train")

# store parameters
logger.print("Parameters")
logger.print("-------------------------")
logger.print(f"Train fraction:    {train_f}")
logger.print(f"Learning rate:     {learning_rate}")
logger.print(f"Batch size:        {batch_size}")
logger.print(f"Hidden layer size: {hid_size}")
logger.print(f"Number of epochs:  {epochs}")
logger.print("-------------------------")

Parameters
-------------------------
Train fraction:    0.9
Learning rate:     0.0001
Batch size:        64
Hidden layer size: 32
Number of epochs:  25
-------------------------


In [31]:
# Read list of the proteins
proteins = []
with open("../../data/lists/train.txt", 'r') as iFile:
    for i in iFile:
        if i[0] != '#':
            proteins.append(i.strip())

In [32]:
train_prots, test_prots = train_test_split(proteins, train_size=train_f, random_state=42)

In [33]:
train_prots = train_prots[:10]

In [34]:
_, feat, charge = read_list(train_prots)
X_train = feat
y_train = charge
_, feat, charge = read_list(test_prots)
X_val = feat
y_val = charge

In [35]:
n_input = feat.shape[1]

In [36]:
y_val

array([2, 2, 2, ..., 2, 2, 2])

# Model

In [37]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(n_input, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(hid_size, activation='relu'),
    tf.keras.layers.Dense(hid_size, activation='relu'),
    tf.keras.layers.Dropout(0.15),
    tf.keras.layers.Dense(hid_size, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(n_cls)
])


In [38]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [39]:
checkpoint_path = "model/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [40]:
model.fit(
         X_train, y_train,
         batch_size=5,
         epochs=5,
         validation_data=(X_val, y_val),
         callbacks=[cp_callback])

Epoch 1/5
15938/15938 [==============================] - ETA: 0s - loss: 0.8877 - accuracy: 0.5765
Epoch 1: saving model to model/cp.ckpt
15938/15938 [==============================] - 48s 3ms/step - loss: 0.8877 - accuracy: 0.5765 - val_loss: 0.9996 - val_accuracy: 0.4923
Epoch 2/5
15920/15938 [============================>.] - ETA: 0s - loss: 0.8535 - accuracy: 0.5935
Epoch 2: saving model to model/cp.ckpt
15938/15938 [==============================] - 46s 3ms/step - loss: 0.8535 - accuracy: 0.5936 - val_loss: 1.0207 - val_accuracy: 0.4936
Epoch 3/5
15934/15938 [============================>.] - ETA: 0s - loss: 0.8366 - accuracy: 0.6024
Epoch 3: saving model to model/cp.ckpt
15938/15938 [==============================] - 48s 3ms/step - loss: 0.8366 - accuracy: 0.6024 - val_loss: 1.0433 - val_accuracy: 0.4860
Epoch 4/5
15926/15938 [============================>.] - ETA: 0s - loss: 0.8286 - accuracy: 0.6074
Epoch 4: saving model to model/cp.ckpt
15938/15938 [===========================